# FastAPI on AWS Lambda with `mangum`

## Overview

This notebook demonstrates how to use the [Mangum](https://github.com/jordaneremieff/mangum) adapter between so that
FastAPI code can be used in an AWS Lambda handler for an API Gateway route.

It converts 

1. API Gateway's JSON representation of HTTP requests format that FastAPI can consume, and
2. FastAPI's HTTP response format into the JSON representation that API Gateway expects.

## Running this notebook

To run this notebook, you will need to 

- `pip install --editable path/to/files-api[test]` your FastAPI app into the venv used by the notebook
- copy/paste `put-audio-file-request.json` so that it is a sibling of this notebook
- copy/paste `get-audio-file-request.json` so that it is a sibling of this notebook

## Install requirements for the Notebook

In [8]:
%%writefile requirements.txt
# this file is generated -- do not edit
mangum
nest_asyncio
rich

Overwriting requirements.txt


In [1]:
! pip install -r requirements.txt &> /dev/null

## Imports

In [3]:
from pathlib import Path
import json

# adapter for FastAPI and lambda handler
from mangum import Mangum

# our API
from files_api.main import create_app
from files_api.settings import Settings

# mock s3
from moto import mock_aws
import boto3

# make mangum's asyncio work in jupyter
import nest_asyncio

# display pretty output
from rich import print
import io
import base64
from IPython.display import Audio, display

ImportError in read_objects.py: No module named 'mypy_boto3_s3'


## Constants

In [4]:
# --- Set these to whatever you like
S3_BUCKET_NAME = "some-bucket"  # can be fake since we're mocking S3
AWS_REGION = "us-east-1"
SAMPLE_APIGW_LAMBDA_PROXY_EVENT_FPATH__PUT = "./put-audio-file-request.json"
SAMPLE_APIGW_LAMBDA_PROXY_EVENT_FPATH__GET = "./get-audio-file-request.json"

# --- Derived constants ---
APP_SETTINGS = Settings(s3_bucket_name=S3_BUCKET_NAME)
FASTAPI_APP = create_app(settings=APP_SETTINGS)

PUT_AUDIO_FILE_EVENT: dict = json.loads(Path(SAMPLE_APIGW_LAMBDA_PROXY_EVENT_FPATH__PUT).read_text())
GET_AUDIO_FILE_EVENT: dict = json.loads(Path(SAMPLE_APIGW_LAMBDA_PROXY_EVENT_FPATH__GET).read_text())


## Utils for Mocking AWS Lambda and S3

Skip over reading this the first time through.

In [9]:
from contextlib import contextmanager
import os
from mangum.types import LambdaContext

# custom context class for Lambda
class MockedLambdaContext(LambdaContext):
    function_name: str = "test_function"
    function_version: str = "1"
    invoked_function_arn: str = f"arn:aws:lambda:{AWS_REGION}:123456789012:function:test_function"
    memory_limit_in_mb: int = 128
    aws_request_id: str = "unique-request-id"
    log_group_name: str = "/aws/lambda/test_function"
    log_stream_name: str = "2021/03/26/[$LATEST]abcdef1234567890abcdef"
    identity: str = None
    client_context: str = None

    def get_remaining_time_in_millis(self) -> int:
        return 30000  # 30 seconds

@contextmanager
def mock_aws_and_env_vars():
    with mock_aws():
        os.environ["AWS_REGION"] = "us-west-2"
        os.environ["AWS_ACCESS_KEY_ID"] = "mock-access-key-id"
        os.environ["AWS_SECRET_ACCESS_KEY"] = "mock-secret-access-key"
        os.environ.pop("AWS_SESSION_TOKEN", None)
        os.environ.pop("AWS_PROFILE", None)
        yield


def display_base64_audio(base64_string: str):
    # Decode the base64 string to binary data
    audio_data = base64.b64decode(base64_string)
    
    # Create an in-memory binary stream
    audio_stream = io.BytesIO(audio_data)
    
    # Display the audio in the notebook
    display(Audio(audio_stream.read(), rate=44100))

## Write and then read a file using the API

Note that in `handler_fn = Mangum(app=FASTAPI_APP, lifespan='off')` we disable the lifespan events
on the FastAPI app.


In [ ]:
# allow nested asyncio event loops; basically makes mangum work in Jupyter
nest_asyncio.apply()

with mock_aws_and_env_vars():
    # create the bucket 
    # print(os.environ["AWS_REGION"])
    boto3.client("s3").create_bucket(
        Bucket=S3_BUCKET_NAME, CreateBucketConfiguration={"LocationConstraint": os.environ["AWS_REGION"]}
    )

    # create a fn(event, context) function from our FastAPI app using the mangum adapter
    handler_fn = Mangum(app=FASTAPI_APP, lifespan='off')

    # pass a JSON event representing `PUT /v1/files/example/generated-speech.mp3`
    response = handler_fn(event=PUT_AUDIO_FILE_EVENT, context=MockedLambdaContext())
    
    print("Response from 'PUT /v1/files/example/generated-speech.mp3'")
    print(response)

    # pass a JSON event representing `GET /v1/files/example/generated-speech.mp3`
    response = handler_fn(event=GET_AUDIO_FILE_EVENT, context=MockedLambdaContext())

    print("Response from 'GET /v1/files/example/generated-speech.mp3' (body omitted because it's looong)")
    response_body: str = response.pop('body')
    print(response)

    display_base64_audio(response_body)


Response from 'PUT /v1/files/example/generated-speech.mp3'

{
    'statusCode': 422,
    'body': '{"detail":[{"type":"missing","loc":["body","file"],"msg":"Field required","input":null}]}',
    'headers': {'content-length': '89', 'content-type': 'application/json'},
    'isBase64Encoded': False
}

Response from 'GET /v1/files/example/generated-speech.mp3' (body omitted because it's looong)

{
    'statusCode': 404,
    'headers': {'content-length': '49', 'content-type': 'application/json'},
    'isBase64Encoded': False
}

Error: Invalid base64-encoded string: number of data characters (37) cannot be 1 more than a multiple of 4